<a href="https://colab.research.google.com/github/gisalgs/notebooks/blob/main/kd-tree-coding-colab.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Coding k-D Trees


At this point, we should be familiar with the concept of k-D trees and we should know how to build a k-D tree given a set of points. If there is any doubt, please view [this video](https://mediasite.osu.edu/Mediasite/Play/4714c558fedc448f880f091f457772b61d) and the related manual exercises in this course module. Now we will work on the actual coding of the k-D tree method. Below we will explain some of the key coding concepts. The complete code for this is available at the github site in a file called [kdtree1.py](https://github.com/gisalgs/indexing/blob/master/kdtree1.py). 

To write Python code for the k-D tree, we first create a class for the nodes in a k-D tree and call it `kDTreeNode`. 

In [ ]:
class kDTreeNode():
    def __init__(self, point, left, right):
        self.point = point
        self.left = left
        self.right = right
    def __repr__(self):
        return str(self.point)

With this class, we can now create a simple tree. We will need to import the Point class to create some points. The use of `!git clone` may be needed to make sure the point.py module in the Jupyter notebook folder. 

In [ ]:
!git clone https://github.com/gisalgs/geom.git 
!git clone https://github.com/gisalgs/indexing.git 

In [ ]:
from geom.point import *

We are going to use the following three points to see how to build a tree through a "manual" process.

In [ ]:
p1 = Point(2, 2)
p2 = Point(0, 5)
p3 = Point(1, 5.5)

We will use `p1` as the root and we will first create the root:

In [ ]:
root = kDTreeNode(p1, None, None)
root

Now, we insert `p2` into the tree. We first compare `p2` with the root using the x-coordinates. Because `p2`'s X coordinate (0) is smaller than that of `p1`'s, `p2` should be the left child node of the root (`p1`). The following code first add `p2` to the tree and the check if it is indeed added.

In [ ]:
root.left = kDTreeNode(p2, None, None)
root.left

Now, we insert `p3`. This time, `p3` is a smaller X coordinate than the root's (`p1`) and therefore it should be in the left branch of the root. However, the left child node of the root is taken by `p2`, we will have to see which branch to go at the node holding `p2`. In order to do so, we alternate the coordinates and now we use the Y coordinates. Since `p3` has a greater Y coordinate than `p2`'s, we go to the right branch where the child node is empty, and we put `p3` there.

In [ ]:
root.left.right = kDTreeNode(p3, None, None)
print(root.left)
print(root.right)
print(root.left.left)
print(root.left.right)

We can actually print the tree and have a visual of it. To do so, we need to import the module of `bst.py` and use the functions there as we have seen before.

In [ ]:
from indexing.bst import *
tree_print(root)

Would it be a surprise that the `tree_print` function can actually print our k-D tree? There are a couple of things to understand this. First, a tree, as we have been exploring so far, is actually represented by its root. So as long as we have a way to get into the tree (through the root, tat is), we have a tree. Second, the `tree_print` function takes a argument of `t` and does not care what it is as long as the input support a few things like `t.left`, `t.right`, and `str(t)`. The first two we clearly have in `kDTreeNode`. The last one is to convert `t` into a string and we have that too (i.e., the `__repr__` method in the class -- ultimately this calls the `__repr__` method of the Point class.

In [ ]:
str(root)

In [ ]:
str(kDTreeNode(Point(100, 101), None, None))

The above experiment works, but it is really tedious. It is like what Cypher says in The Matrix: what a mind job! 

We can make our life a little easier by writing a function that can tell which direction (left or right) to go given a node and a point object. We do this in the `kdcompare` function as listed below. This function takes a third argument that is the depth of the node. The depth of a node is how far down the tree it is from the root, as measured in the number of edges on the path from that node to the root. As we know, the depth of a node tells us which dimension (X or Y) to use. At depth 0 (root) we use the X coordinate, at depth 1 we use Y, depth 2 X, and so on. 

In [ ]:
def kdcompare(r, p, depth):
    k = len(p)
    dim = depth%k
    if p[dim] <= r.point[dim]:  # left branch
        return -1
    else:                       # right branch
        return 1

The above code relies on a special Python `%` operator that is related to division. It is the [modulo operator](https://docs.python.org/3/reference/expressions.html) or modulus operator, which yields the remainder of the division between two numbers:

In [ ]:
print(100 % 3)
print(2 % 3)
print(1.5 % 3)

Here, we can test if an integer is even or odd by checking if the remainder of dividing that number by 2. If the remainder is 0, it is an even number, otherwise, the remainder will be 1 and it is an odd number.

In [ ]:
[i%2 for i in range(10)] # the remainders of each of the numbers from 0 to 9 divided by 2

In our `Point` class, we override two important methods, [`__getitem__`](https://github.com/gisalgs/geom/blob/132ab6d6529a3431faf03085f2c0717f811ece09/point.py#L36) and [`__len__`](https://github.com/gisalgs/geom/blob/132ab6d6529a3431faf03085f2c0717f811ece09/point.py#L40), which allow us to use the brackets to get the X or Y coordinates by specifying the index to be 0 or 1, respectively. Below are a few examples that show how this works:

In [ ]:
print(p2[0], p2[1])
print(p2.x, p2.y)

So given the depth of a node and a point, we can know which coordinate to use as shown in the second line of function `kdcompare`, where `dim` is either 0 (X) or 1 (Y). Here is an example that mimics what we have in the function on a new point.

In [ ]:
p4 = Point(0, 4)
k = len(p4) # always return 2 as specified in point.py
depth = 1
dim = depth%k
print(dim)

When we insert `p4` into the tree, we first look at the root and use the `kdcompare` function as follows:

In [ ]:
kdcompare(root, p4, 0)

Now we know we should go to the left child node of the root. Then we need to test if the left child node is taken or not:

In [ ]:
root.left == None

Apparently, the above test tells us it is the left child node is not empty. This means we have to go down one more level to see if we can put `p3` there. This time we use the function again but with a different node and depth:

In [ ]:
kdcompare(root.left, p4, 1)

Now we know `p4` should be put in the left child node (because the function returns -1) of `root.left`, if the child node is empty (otherwise we need to go further down). We can test it: 

In [ ]:
root.left.left == None

Yes! It is empty and we can simply put `p4` there as a new node:

In [ ]:
root.left.left =kDTreeNode(p4, None, None)
tree_print(root)

The process can continue whenever we have more points to be inserted into the tree. But of course we know that we won't be doing all of that manually like this. Instead, we write a function to take care of this.

One of the tedious things we have seen from above is that we have to keep tracking down the tree in order to find the right place for the node to be placed (i.e., finding the parent node of the new point). We will write a function that can actually do that automatically as long as we provide the root and the point. We will want this function to do two things at a time: to either find the parent node of a point, or to find if the point is in the tree. In the following function, the four parameters and the [source code](https://github.com/gisalgs/indexing/blob/3136b8580d8a9354f4667dfcd3a2c9c80d7e2376/kdtree1.py#L85) at the github site has the docstring to explain these parameters. This function is also similar to what we have seen in the [search_bt](https://github.com/gisalgs/indexing/blob/3136b8580d8a9354f4667dfcd3a2c9c80d7e2376/bst.py#L20) function in bst.py.

Similar to the binary search tree we discussed before, we need a function that can query (search) the tree, and we call it `query_kdtree`. The main difference here is the use of a `depth` parameter in the `query_kdtree` function to track the depth of search, which allows us to determine which dimension, X or Y, should be used. In the version of `query_kdtree` below, we return two values instead of one, to indicate both the node and whether the new data should be placed in the left or right child node of the returned node (only when `is_find_only` is set to False).


In [ ]:
def query_kdtree(t, p, depth=0, is_find_only=True):
    if t is None:
        return None, None
    if t.point == p:
        if is_find_only:
            return t, None
        else:
            return None, None
    lr = kdcompare(t, p, depth)
    if lr<0:
        child = t.left
    else:
        child = t.right
    if is_find_only==False and child is None:
        return t, lr
    return query_kdtree(child, p, depth+1, is_find_only)

We can test the above function using our tree (`root`) and `p4`. But since we have already inserted `p4` so it should return the point as part of the returned tuple.

In [ ]:
query_kdtree(root, p4)

If we want to insert `p4` again by trying to find its parent node, the function will return `None, None` since we don't want to store duplicated information in the tree:

In [ ]:
query_kdtree(root, p4, is_find_only=False)

We can further test it by "removing" `p4` from the tree and do this again:

In [ ]:
root.left.left = None
tree_print(root)

print('\n', '-'*20, '\n', sep='')

parent, left_or_right = query_kdtree(root, p4, is_find_only=False)
if parent != None:
    new_node = kDTreeNode(p4, None, None)
    if left_or_right < 0:
        parent.left = new_node
    else:
        parent.right = new_node
tree_print(root)

## <font color="red">Question 1</font>

Function `query_kdtree` is recursive. What are the conditions that stop the recursive calls? In other words, what are the base cases for the recursion?


```python
# TODO
#
#    Double click on this cell and write your answer below. 
```





Finally, we write a function called `kdtree` that can be used to create a k-D tree using a list of points. The points will be inserted into the tree using exactly how they are presented in the list (from left to right). The first point in the list will be used as the root. 

In [ ]:
def kdtree(points):
    root = kDTreeNode(point=points[0], left=None, right=None)
    for p in points[1:]:
        node = kDTreeNode(point=p, left=None, right=None)
        p0, lr = query_kdtree(root, p, 0, False)
        if p0 is None and lr is None:   # skip if duplicated
            continue
        if lr<0:
            p0.left = node
        else:
            p0.right = node
    return root

Now, we test everything we have got so far using the same four points we have been using. This time, we are actually going to give the tree a better variable name as in `tree1` instead of calling it `root`. But we should know that in our representation of the tree, we effectively use the root node as the tree. This is because we alway start from the root with we deal with a tree.

In [ ]:
tree1 = kdtree([p1, p2, p3, p4])
tree_print(tree1)

All of the above code is in a file called **kdtree1.py** that is available on github at [here](https://github.com/gisalgs/indexing/blob/master/kdtree1.py). We need to put it in the **indexing** folder. This folder is at the same level of the **geom** folder we have used before. Also make sure to have an blank file called `__init__.py` in the indexing folder.

Notes:

1. The functions listed above may be different from the textbook. Function `query_kdtree` is where most of the updates were made since the book was first published. The latest versions are available on github.
2. The `kdtree` function is now safe to run on a list with duplicated points. The original version (i.e., the one on the text book) would raise an exception (by Python) when there are duplicated points in the list.

We can test the code using the following (or similar) lines of code.


In [ ]:
from geom.point import *
from indexing.kdtree1 import *

help(query_kdtree)

In [ ]:
data1 = [ (2,2), (0,5), (8,0), (9,8), (7,14), (13,12), (14,13) ]
points = [Point(d[0], d[1]) for d in data1]

t1 = kdtree(points)

print([query_kdtree(t1, p)[0] for p in points])
print(points)

In [ ]:
tree_print(t1)

In [ ]:
bt_print(t1)

## <font color="red">Question 2</font>

As we see from the above, the `bt_print` function in `bst.py` can be used to print out any binary tree. Explain the order of the print out and why this function does not print out the points as they entered into the tree. Please type your answer in the following cell. To anser this question, it will be necessary to carefully read the function in [bst.py](https://github.com/gisalgs/indexing/blob/master/bst.py) to understand how this function works.


```python
# TODO
#
#    Double click on this cell and write your answer below. 
```




## Balancing k-D trees

The tree created using the above code is not balanced unless the predefined order can lead to a balanced tree (which of course is hard to do). Now, instead of always choosing the next point in the list to insert, we always choose the median point at the corresponding X or Y coordinate and insert this median point in to the tree. We implement this idea in the following function called `kdtree2`.


In [ ]:
def kdtree2(points, depth = 0):
    if len(points)==0:
        return
    k = len(points[0])
    axis = depth % k
    points.sort(key=lambda p: p[axis])
    pivot = len(points)//2
    return kDTreeNode(point=points[pivot],
                      left=kdtree2(points[:pivot], depth+1),
                      right=kdtree2(points[pivot+1:], depth+1))

In [ ]:
t2 = kdtree2(points)
tree_print(t2)

The above function is in the same file [`kdtree1.py`](https://github.com/gisalgs/indexing/blob/master/kdtree1.py) as before. We will give it a test below. 

The important part here is the use of the **sort** method that comes with any list. The sort method has an option called `key`, which allows to specify the values to be sorted. In our case, sometimes we need to sort the list based on the X coordinates of the points (odd depths), but sometimes we need to sort based on Y coordinates (even depths). For example, when we want to sort a list points using the Y coordinates, we can define a separate function that returns the Y coordinate of a given point and use it in the sort:

In [ ]:
pts = [p1, p2, p3, p4]
print('original order:', pts)

def use_y(p):
    return p[1]

def use_x(p):
    return p[0]

pts.sort(key=use_y)
print('sorted by Y:', pts)

pts.sort(key=use_x)
print('sorted by X:', pts)

This is works but is a little clumsy. There are packages like itemgetter or attrgetter. But there is an elegant solution without using more packages. This is where the **lambda functions** come in. 

### Lambda functions

In Python, lambda functions are a quick way to make a nameless function that will be only called upon when needed; for this reason, we can also call lambda functions throw-away functions. This function right now doesn't handle duplicated points in the list. Sometimes, we need a function but don’t need a real function with the name. This is an anonymous function and we use the Python construct lambda for this purpose. For example, below we define a function so that it returns True if an input is an even number or False otherwise. This is a quick way of defining a function without using the def keyword and all the syntax rules. It is quite simple: the variables before the colon are the inputs (arguments) and everything after the colon is an expression that will be evaluated and returned:

```python
lambda arguments: expression
```

Here is a simple example:

In [ ]:
f = lambda x: x%2==0
f(10)

In [ ]:
f(1)

Lambda functions provide great expressiveness and abstraction of our code. Now we demonstrate its use in the context of list sorting of points, which cannot be readily done using the built-in sort function.

In [ ]:
pts = [p1, p2, p3, p4]
pts.sort(key=lambda p: p[1])
print('sorted by Y:', pts)

Let's see what exactly happens by changing the code so we print out the details in every iterations. We should be able to observe that the code keeps dividing the list of points into halves and sorting the points in those halves using the proper X or Y coordinates.

In [ ]:
def kdtree2x(points, depth = 0, side='none'):
    if len(points)==0:
        return
    k = len(points[0])
    axis = depth % k
    points.sort(key=lambda p: p[axis])
    pivot = len(points)//2
    print('depth', depth, side, end=': ')
    print(' '.join([str(p) for p in points[:pivot]]), end=' ')
    print('*' + str(points[pivot]) + '*', end=' ')
    print(' '.join([str(p) for p in points[pivot+1:]]))
    return kDTreeNode(point=points[pivot],
                      left=kdtree2x(points[:pivot], depth+1, 'left'),
                      right=kdtree2x(points[pivot+1:], depth+1, 'right'))


In [ ]:
t2a = kdtree2x(points)
print()
tree_print(t2a)

## <font color="red">Question 3</font>

When we use `kdtree2` to generate `t2`, is the point at (13, 12) the third point entered to the tree? Why or why not? If it is not, how many points are entered to the tree before this point? Note that one way to find out is to slightly modify the code to print out information needed to answer this question. The function `kdtree2x` is a good example of changing the code for our own experiments. You can copy and past the code of `kdtree2` and change to another name and then run it to test. Add more cells here if necessary to anser this question. 

You can also answer this question by just examine the code without running it. As soon as you get the idea of how this function works, the answer should be clear by then.

```python
# TODO
#
#    Double click on this cell and write your answer below. 
```


